In [1]:
# config
config = {
    "data_path": "../",
    "model": {
        "loss_smooth": 1.0,
        "optimizer_params": {"lr": 0.001, "weight_decay": 0.0},
        "scheduler": {
            "name": "CosineAnnealingLR",
            "params": {
                "CosineAnnealingLR": {"T_max": 50, "eta_min": 1e-06, "last_epoch": -1},
                "ReduceLROnPlateau": {
                    "factor": 0.316,
                    "mode": "min",
                    "patience": 3,
                    "verbose": True,
                },
            },
        },
    },
    "output_dir": "models",
    "progress_bar_refresh_rate": 500,
    "seed": 42,
    "train_bs": 64,
    "trainer": {
        "enable_progress_bar": True,
        "max_epochs": 30,
        "min_epochs": 20,
        "accelerator": "mps",
        "devices": 1,
    },
    "valid_bs": 64,
    "workers": 0,
    "device": "mps",
    "folds": {
        "n_splits": 4,
        "random_state": 42,
        "train_folds": [0, 1, 2, 3]
    }
}

import pandas as pd
from process_data import DataProcessor

xtrain = pd.read_csv('data/single_turbine_data/X_train.csv')
xtest = pd.read_csv('data/single_turbine_data/X_test.csv')
ytrain = pd.read_csv('data/single_turbine_data/y_train.csv')
ytest = pd.read_csv('data/single_turbine_data/y_test.csv')

# combine train and test
X_train = pd.concat([xtrain, xtest])
y_train = pd.concat([ytrain, ytest])

processor = DataProcessor(X_train)
processor.process_all()
X_train = processor.data

label = ['1_Gear oil temperature (°C)']
y_train = y_train[label]

original_cols = [
    # '1_Wind direction (°)',
    #    '1_Nacelle position (°)', 
       'avg_dir',
    #    '1_Power (kW)',
    '1_Wind speed (m/s)',
       '1_Front bearing temperature (°C)', '1_Rear bearing temperature (°C)',
    #    '1_Stator temperature 1 (°C)', 
    #    '1_Nacelle ambient temperature (°C)',
       '1_Nacelle temperature (°C)', 
    #    '1_Transformer temperature (°C)',
    #    '1_Generator bearing rear temperature (°C)',
       '1_Generator bearing front temperature (°C)', 
    #    '1_Temp. top box (°C)',
    #    '1_Hub temperature (°C)', 
       '1_Ambient temperature (converter) (°C)',
       '1_Rotor bearing temp (°C)', 
       '1_Transformer cell temperature (°C)', 
    #    '1_Generator RPM (RPM)',
       ]
leadsnlags = [
       '1_Wind direction (°)_lead6', 
       '1_Nacelle position (°)_lead3',
       '1_Power (kW)_lag6', 
       '1_Stator temperature 1 (°C)_lag1',
       '1_Nacelle ambient temperature (°C)_lead6',
       '1_Transformer temperature (°C)_lead6',
       '1_Generator bearing rear temperature (°C)_lag1',
       '1_Temp. top box (°C)_lag1', 
       '1_Hub temperature (°C)_lead6',
       '1_Ambient temperature (converter) (°C)_lead6',
       '1_Transformer cell temperature (°C)_lead6',
       '1_Generator RPM (RPM)_lead6']
extras = [
    # 'curtailed', 
    'offline',
    'historic_offline',
    ]

mean = [col for col in X_train.columns if 'mean' in col]
std = [col for col in X_train.columns if 'std' in col]
crest = [col for col in X_train.columns if 'crest' in col]
# kurt = [col for col in X_train.columns if 'kurt' in col]
change = [col for col in X_train.columns if 'change' in col]

cols = original_cols + extras + leadsnlags + mean + std + crest
X_train = X_train[cols]

# scale the data
from sklearn.preprocessing import StandardScaler, RobustScaler
scalerx = RobustScaler()
scalery = RobustScaler()
X_train = scalerx.fit_transform(X_train)
y_train = scalery.fit_transform(y_train)


/Users/johnny/anaconda3/envs/kaggle/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/johnny/anaconda3/envs/kaggle/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/johnny/anaconda3/envs/kaggle/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/johnny/anaconda3/envs/kaggle/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and wil

In [2]:
X_train.shape

(210937, 74)

In [3]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar
import lightning.pytorch as pl
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau

# Convert Pandas DataFrame to PyTorch Tensor
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)

# Create DataLoader
# train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)


# Define Neural Network
class LightningModule(pl.LightningModule):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.config = config
        self.layer1 = nn.Linear(input_dim, input_dim*2)
        self.layer2 = nn.Linear(input_dim*2, input_dim*2)
        self.layer3 = nn.Linear(input_dim*2, output_dim)
        # self.relu = nn.ReLU()
        self.relu = nn.LeakyReLU(0.3)
        # self.dropout = nn.Dropout(p=0.2)
        self.loss = nn.MSELoss()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        # x = self.dropout(x)
        x = self.relu(self.layer2(x))
        # x = self.dropout(x)
        x = self.layer3(x)
        return x

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters())

        if self.config['model']["scheduler"]["name"] == "CosineAnnealingLR":
            scheduler = CosineAnnealingLR(
                optimizer,
                **self.config['model']["scheduler"]["params"][self.config['model']["scheduler"]["name"]],
            )
            lr_scheduler_dict = {"scheduler": scheduler, "interval": "step"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler_dict}
        elif self.config['model']["scheduler"]["name"] == "ReduceLROnPlateau":
            scheduler = ReduceLROnPlateau(
                optimizer,
                **self.config['model']["scheduler"]["params"][self.config['model']["scheduler"]["name"]],
            )
            lr_scheduler = {"scheduler": scheduler, "monitor": "val_loss"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat, y)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        for param_group in self.trainer.optimizers[0].param_groups:
            lr = param_group["lr"]
        self.log("lr", lr, on_step=False, on_epoch=True, prog_bar=True)
        logs = {"train_loss": loss, "lr": lr}
        return {"loss": loss, "log": logs}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat, y)
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)
        return {"val_loss": loss}


from lightning.pytorch.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="my_model")
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from math import sqrt

# Prepare the KFold cross-validation
kf = KFold(n_splits=config["folds"]["n_splits"])

# Initialize results DataFrame
overall_results = pd.DataFrame()

# Main loop for K-Fold Cross-Validation
for fold, (train_index, val_index) in enumerate(kf.split(X_train, y_train)):
    print(f"Training Fold {fold+1}")
    # Subset the data
    X_train_fold = X_train[train_index]
    y_train_fold = y_train[train_index]
    X_val_fold = X_train[val_index]
    y_val_fold = y_train[val_index]

    # Create DataLoader for this fold
    train_dataset = TensorDataset(torch.FloatTensor(X_train_fold), torch.FloatTensor(y_train_fold))
    train_loader = DataLoader(train_dataset, batch_size=config["train_bs"], shuffle=True)

    val_dataset = TensorDataset(torch.FloatTensor(X_val_fold), torch.FloatTensor(y_val_fold))
    val_loader = DataLoader(val_dataset, batch_size=config["valid_bs"], shuffle=True)

    # Initialize model
    model = LightningModule(X_train.shape[1], y_train.shape[1])

    early_stop_callback = EarlyStopping(monitor="val_loss", mode="min", patience=5, verbose=1)
    progress_bar_callback = TQDMProgressBar(refresh_rate=config["progress_bar_refresh_rate"])

    checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min",
    save_top_k=1,
    verbose=1,
    dirpath=config["output_dir"],
    filename="best_model_fold_" + str(fold + 1),
)
    
    # Initialize trainer
    trainer = pl.Trainer(
        callbacks=[early_stop_callback, progress_bar_callback, checkpoint_callback],
        logger=logger,
        **config["trainer"]
    )

    # Train the model on this fold
    trainer.fit(model, train_loader, val_loader)

    # Validation
    model.eval()
    y_pred_fold = model(torch.FloatTensor(X_val_fold)).detach().numpy()

    # Inverse scaling for validation data and prediction
    y_val_fold = scalery.inverse_transform(y_val_fold)
    y_pred_fold = scalery.inverse_transform(y_pred_fold)

    # Evaluate the model on this fold
    rmse_fold = sqrt(mean_squared_error(y_val_fold, y_pred_fold))
    print(f'Validation RMSE for Fold {fold+1}: {rmse_fold}')

    # Save results for this fold
    results_df = pd.DataFrame(
        {
            'Model': f'Neural Network - Fold {fold+1}',
            'Validation RMSE': rmse_fold,
            'Iterations': trainer.global_step,
            'Learning Rate': config['model']['optimizer_params']['lr'],
            'Depth': [X_train.shape[1], 64, 64, y_train.shape[1]],
            'Loss Function': 'MSE',
            'Features': ', '.join(cols),
        }
    )
    overall_results = pd.concat([overall_results, results_df])

    # save model
    # torch.save(model.state_dict(), f"models/nn_fold{fold+1}.pth")

from pathlib import Path
# Save results
results_file = Path('results.csv')
# Save overall results
if results_file.exists():
    existing_df = pd.read_csv(results_file)
    overall_results = pd.concat([existing_df, overall_results])

overall_results.to_csv(results_file, index=False)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Training Fold 1


/Users/johnny/anaconda3/envs/kaggle/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:622: UserWarning: Checkpoint directory /Users/johnny/Library/CloudStorage/OneDrive-Personal/py/RES_hackathons/Temp Sensor/models exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name   | Type      | Params
-------------------------------------
0 | layer1 | Linear    | 11.1 K
1 | layer2 | Linear    | 22.1 K
2 | layer3 | Linear    | 149   
3 | relu   | LeakyReLU | 0     
4 | loss   | MSELoss   | 0     
-------------------------------------
33.3 K    Trainable params
0         Non-trainable params
33.3 K    Total params
0.133     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/johnny/anaconda3/envs/kaggle/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:483: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
/Users/johnny/anaconda3/envs/kaggle/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:438: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/johnny/anaconda3/envs/kaggle/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:438: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` ar

...

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Metric val_loss improved. New best score: 0.009
Epoch 0, global step 2472: 'val_loss' reached 0.00864 (best 0.00864), saving model to '/Users/johnny/Library/CloudStorage/OneDrive-Personal/py/RES_hackathons/Temp Sensor/models/best_model_fold_4-v15.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

Metric val_loss improved by 0.001 >= min_delta = 0.0. New best score: 0.008
Epoch 1, global step 4944: 'val_loss' reached 0.00791 (best 0.00791), saving model to '/Users/johnny/Library/CloudStorage/OneDrive-Personal/py/RES_hackathons/Temp Sensor/models/best_model_fold_4-v15.ckpt' as top 1


* 0.438, 0.524, 0.569, 0.523

* 0.465, 0.471, 0.465, 0.505 window 6 centre true
* 0.422, 0.452, 0.437, 0.492 window 12 false
* , 0.53, 0.486, 0.484 window 50
* 0.415, 0.478, 0.428, 0.418 window 20
* , , 0.447, 0.440 window 30
* , 0.488, 0.436, 0.482 window 25
* , 0.448, 0.444, 0.491  window 15